# Common Section
Functions and Function usage examples are in the two cells that follow

In [2]:
# For your development, copy all code from this cell and put it in your ipynb
# As far as I've read, there's no simple way of directly sharing code
#   across .ipynb notebooks; which really sucks.

using Images, DataFrames, FixedPointNumbers, PyPlot, Colors

# loads all jpg image files inside the directory
function get_image_files(image_dir)
    # get all the files in the directory
    dir_contents = readdir(image_dir)
    dir_contents = [ join([image_dir, dir_contents[i]], "/") for i in 1:length(dir_contents) ]
    
    # retain only jpg image files
    image_files = filter(x->ismatch(r".jpg$", x), dir_contents)
    
    # return the paths to the jpg image files
    return image_files
end

# converts image to a h x w matrix
#   h = height
#   w = width
function convert_image_to_mat(image)
    image_mat = reinterpret(UInt8, separate(image).data)
    return image_mat
end

# converts a h x w matrix to an Images module image
function convert_mat_to_image(image_mat)
    image = colorim(image_mat)
    return image
end

# load a list of image files as a cell array of image matrices
function load_images_as_cellarray_mats(image_files)
    images_mat = cell(length(image_files),1)
    for (image_n, image_file) in enumerate(image_files)
        image = load(image_file)
        image_mat = convert_image_to_mat(image)
        images_mat[image_n] = image_mat
    end
    return images_mat
end

# convert a cell array of image matrices to a cell array of images
function load_cellarray_mats_as_images(images_mat)
    images = cell(length(images_mat),1)
    for (image_n, image_mat) in enumerate(images_mat)
        image = convert_mat_to_image(image_mat)
        images[image_n] = image
    end
    return images
end

; # suppress display

In [4]:
# function usage examples summary
#   if you write any new functions in your code, please maintain two such cells
#   e.g. like cell above, one cell with ALL the functions
#        like this cell, a summary of all the function that you have written, with an example syntax

# image_files = get_image_files("./beer1800")
# images_mat = load_images_as_cellarray_mats(image_files)
# images = load_cellarray_mats_as_images(images_mat)
# image_mat = convert_image_to_mat(images[1])
# image = convert_mat_to_image(image_mat)

# Individual Section
At the moment the cells that follow just have dummy code. I will be using the cells below for my own experiments.

My suggestion - Let's copy the code in the first two cells (common section) and do our own experiments in separate files. We can meet regularly and decide what functions that we write will be ported to the final submission and merge it accordingly.

Another suggestion - I've named this file as **main\_ananth.ipynb**. If you guys also follow the same convention (**main\_\<name\>.ipynb**) for consistency, it would be great. Also, this way, we can name our final code as **main.ipynb** and rename it later as we see fit.

In [ ]:
image_sizes = zeros(length(image_files), 2)
for (index, image_file) in enumerate(image_files)
    image = load(image_file)
    image_sizes[index,:] = collect(size(image))
end

In [ ]:
# figure(1)
# grid("on")
# plot(image_sizes[:,1], image_sizes[:,2], "o")
# xlabel("width of images")
# ylabel("height of images")
println(minimum(image_sizes[:,2]))
println(maximum(image_sizes[:,2]))